In [1]:
# Jade Chongsathapornpong, July 2023
from os.path import join
from os import getcwd

import numpy as np
import matplotlib.pyplot as plt

### Please note actual usage is pending an update to struct2XAS to fix an
### issue that appeared during https://github.com/xraypy/xraylarch/pull/458

from larch.xafs import FeffRunner
# from larch.xrd.struct2xas import Struct2XAS
from struct2xas import Struct2XAS # replace this with the above eventually

# Generate FDMNES input

In [2]:
root = r"C:\Users\jade2\Dropbox (Personal)\Jade_CNRS_Internship\Data\CIF_references"
template = r"C:\Users\jade2\Dropbox (Personal)\Jade_CNRS_Internship\heo-xas-xrd\fdmnes.tmpl" # for cluster calculation
template_test = r"C:\Users\jade2\Dropbox (Personal)\Jade_CNRS_Internship\heo-xas-xrd\fdmnes_test.tmpl" # for local calculation

# namesZnO_W = ['EntryWithCollCode157724', 'EntryWithCollCode154486'] # ICSD
# namesZnO_R = ['EntryWithCollCode190807', 'EntryWithCollCode57156'] # ICSD
# namesCuO_T = ['COD4105682', 'COD9008961']
# namesNiO_R = ['EntryWithCollCode9866']

# namesCo3O4 = ['COD9005892', 'COD1526734']
# namesCoO = ['COD1528838', 'COD1533087']
# ZnCo2O4 = 'COD5910136'
Co15Zn85O = 'COD1533020'

CoCoSpinel = 'EntryWithCollCode24210'
CuCoSpinel = 'EntryWithCollCode36356_modif'
# MgCoSpinel = '' # seems to have the same issue as that one other spinel
NiCoSpinel = 'EntryWithCollCode181262'
Zn037Co263Spinel = 'EntryWithCollCode261867' # seems to crash struct2XAS when I try to get Zn. It also doesn't seem to recognize partial occupancy correctly.
ZnCo2O4Spinel_partialocc = 'EntryWithCollCode73758'
ZnCo2O4Spinel_mod = 'EntryWithCollCode5974'
ZnOBlende = 'EntryWithCollCode647683'
# ZnOWurtz = r'Spinels\ZnO_W\EntryWithCollCode154486' # already done
# ZnORock = r'Spinels\ZnO_R\EntryWithCollCode190807' # already done

In [4]:
# Be sure to delete fdmfile before making the inputs for a new batch calculation (including between testing/final)!

mode = 0 # 0 = no output ; 1 = test output; 2 = final output; 3 = both test and final
elem = 'Co'
outpath_test = 'local_FDMNES/Spinels'
outpath = 'FDMNES/Spinels'
infilename = CoCoSpinel
outfilename = 'ZnOBlende_B-Td'

path = join(root, infilename + '.cif')

s = Struct2XAS(path, elem)
sites = s.get_abs_sites()
print(sites)
s.set_abs_site(0) # corresponds to abs_index, the first element printed from the line above

s.visualize(radius=7)

if mode != 0 and input("Continue and write files? (y/n)") == 'y':
    if mode == 1:
        s.make_input_fdmnes(4, template=template_test, output_path=outpath_test, name=outfilename, green=True)
    elif mode == 2:
        s.make_input_fdmnes(7, template=template, output_path=outpath, name=outfilename, green=False)
    elif mode == 3:
        s.make_input_fdmnes(4, template=template_test, output_path=outpath_test, name=outfilename, green=True)
        s.make_input_fdmnes(7, template=template, output_path=outpath, name=outfilename, green=False)

[struct2xas] INFO : Frames: 1, Absorbing sites: 2. (Indexes for frames and abs_sites start at 0)
[[0, 'Co2+', [0.125, 0.125, 0.125], '8b', array([1.0104, 1.0104, 1.0104]), 1, 1], [1, 'Co3+', [0.0, 0.0, 0.5], '16c', array([0.    , 0.    , 4.0418]), 1, 23]]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

[struct2xas] INFO : {'Co': 'red', 'O': 'green'}


In [ ]:
def plot_job(path, label):
    arr = np.genfromtxt(path, skip_header=4).transpose()
    plt.plot(arr[0], arr[1], label=label)

plot_job('FDMNES/5-cation_solidsolution/Cu/job_conv.txt', 'Cu(R)')
# plt.plot(arr[0], arr[1], label='Cu (R)')
# arr2 = np.genfromtxt('FDMNES/CuO_Tenorite/job_conv.txt', skip_header=4).transpose()
# plt.plot(arr2[0], arr2[1], label='Cu (T)')
# arr3 = np.genfromtxt('FDMNES/5-cation_solidsolution/Co/job_conv.txt', skip_header=4).transpose()
# plt.plot(arr3[0], arr3[1], label='Co (R)')
# arr = np.genfromtxt('FDMNES/CuO_Tenorite/directcif/job.txt', skip_header=4).transpose()
# plt.plot(arr[0], arr[1], label='using CIF directly')
# arr1 = np.genfromtxt('FDMNES/CuO_Tenorite/s2x/job_s2x.txt', skip_header=4).transpose()
# plt.plot(arr1[0], arr1[1], label='using struct2XAS pre-gen file')

arr = np.genfromtxt('FDMNES/COD4105682.txt', skip_header=4).transpose()
arr1 = np.genfromtxt('../Data/BM23_Averaging_PyMCA/Cu_Kedge/CuO_Aldritch_RT.csv', delimiter=',').transpose()
print(arr.shape)
print(arr1.shape)
plt.plot(arr[0], arr[1], label='CuO (T) FDMNES')
plt.plot(arr1[0]*1000 - 8984, arr1[1] * 6, label='CuO Aldritch (T) BM23')
plt.xlim(-30,250)
plt.legend()
plt.savefig('Output/CuO_T', dpi=150)
plt.show()

# Generate FEFF input and run FEFF

In [ ]:
feff_temp = 'feff_exafs.tmpl'
s.make_input_feff(radius=7, template=feff_temp, parent_path='local_FEFF')
feff_inp = 'local_FEFF/feff.inp'

In [ ]:
sim = FeffRunner(feff_inp)
sim.run()

In [ ]:
# plot FEFF output